In [5]:
import pandas as pd
import numpy as np

import sys
import yaml
from pathlib import Path

sys.path.append("../")
sys.path.append("../../modules/yolov5/")
from reef.yolo.data import LoadImagesAndLabels

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [7]:
SLICE_HEIGHT = 360
SLICE_WIDTH = 640
IMG_SIZE = 640
STRIDE = 32
RANDOM_STATE = 42
OVERLAP_THRESHOLD = 0.25

In [10]:
split_path = Path("../../data/splits/s50/")

with open(split_path / "data.yaml") as f:
    data = yaml.load(f, yaml.Loader)
    
with open("../reef/yolo/hyp.yaml") as f:
    hyp = yaml.load(f, yaml.Loader)
    
data

{'names': ['starfish'],
 'nc': 1,
 'train': '/home/isabella/code/competitions/great_barrier_reef/data/splits/s50/train.txt',
 'val': '/home/isabella/code/competitions/great_barrier_reef/data/splits/s50/valid.txt'}

In [40]:
train_dataset = LoadImagesAndLabels(
    path=data['train'],
    slice_height=SLICE_HEIGHT,
    slice_width=SLICE_WIDTH,
    overlap_threshold=OVERLAP_THRESHOLD,
    img_size=IMG_SIZE,
    augment=True,
    stride=STRIDE,
    random_state=RANDOM_STATE,
    hyp=hyp,
    prefix="train: ",
)

valid_dataset = LoadImagesAndLabels(
    path=data['val'],
    slice_height=SLICE_HEIGHT,
    slice_width=SLICE_WIDTH,
    overlap_threshold=OVERLAP_THRESHOLD,
    img_size=IMG_SIZE,
    augment=True,
    stride=STRIDE,
    random_state=RANDOM_STATE,
    hyp=hyp,
    prefix="valid: ",
)

# Model

In [31]:
import torch
from models.yolo import Model
from utils.general import intersect_dicts

# load model
nc = data['nc']
device = 'cpu'


ckpt = torch.load("../../weights/yolov5s.pt")

model = Model(
    ckpt['model'].yaml, 
    ch=3, 
    nc=nc, 
    anchors=hyp.get('anchors')
).to(device)
# exclude keys
exclude = ['anchor'] if (hyp.get('anchors')) and not resume else []
# checkpoint state_dict as FP32
csd = ckpt['model'].float().state_dict()
# intersect
csd = intersect_dicts(csd, model.state_dict(), exclude=exclude)
model.load_state_dict(csd, strict=False)

12/15/2021 23:02:39 - INFO - utils.general -   Overriding model.yaml nc=80 with nc=1
12/15/2021 23:02:39 - INFO - utils.general -   
                 from  n    params  module                                  arguments                     
12/15/2021 23:02:39 - INFO - utils.general -     0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
12/15/2021 23:02:39 - INFO - utils.general -     1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
12/15/2021 23:02:39 - INFO - utils.general -     2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
12/15/2021 23:02:39 - INFO - utils.general -     3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
12/15/2021 23:02:39 - INFO - utils.general -     4                -1  2    115712  models.common.C3                        [128, 128, 2]        

_IncompatibleKeys(missing_keys=['model.24.m.0.weight', 'model.24.m.0.bias', 'model.24.m.1.weight', 'model.24.m.1.bias', 'model.24.m.2.weight', 'model.24.m.2.bias'], unexpected_keys=[])

In [77]:
del model, train_dataloader, train_dataset, valid_dataset

[autoreload of reef.yolo.lightning failed: Traceback (most recent call last):
  File "/home/isabella/code/competitions/great_barrier_reef/env/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/isabella/code/competitions/great_barrier_reef/env/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/opt/anaconda3/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/opt/anaconda3/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "../reef/yolo/lightning.py", line 19, in <module>
    from yolo.data import LoadImagesAndLabels
ModuleNotFoundErro

## Freeze layers

In [32]:
freeze_layers = 0

freeze = [f'model.{x}.' for x in range(freeze_layers)]
for k, v in model.named_parameters():
    v.requires_grad = True
    if any(x in k for x in freeze):
        v.requires_grad = False

## Optimizer

In [34]:
from torch.optim import Adam, SGD
from torch import nn

adam = True
batch_size = 8

# Optimizer
nbs = 64
accumulate = max(round(nbs / batch_size), 1)
hyp['weight_decay'] *= batch_size * accumulate / nbs
print("Scaled weight decay", hyp['weight_decay'])

g0, g1, g2 = [], [], []  # optimizer parameter groups
for v in model.modules():
    if hasattr(v, 'bias') and isinstance(v.bias, nn.Parameter):  # bias
        g2.append(v.bias)
    if isinstance(v, nn.BatchNorm2d):  # weight (no decay)
        g0.append(v.weight)
    elif hasattr(v, 'weight') and isinstance(v.weight, nn.Parameter):  # weight (with decay)
        g1.append(v.weight)

if adam:
    optimizer = Adam(g0, lr=hyp['lr0'], betas=(hyp['momentum'], 0.999))  # adjust beta1 to momentum
else:
    optimizer = SGD(g0, lr=hyp['lr0'], momentum=hyp['momentum'], nesterov=True)
    
optimizer.add_param_group({'params': g1, 'weight_decay': hyp['weight_decay']})
optimizer.add_param_group({'params': g2})

del g0, g1, g2

Scaled weight decay 0.0005


## Learning rate scheduler

In [36]:
import math
from torch.optim import lr_scheduler

def one_cycle(y1=0.0, y2=1.0, steps=100):
    # lambda function for sinusoidal ramp from y1 to y2 https://arxiv.org/pdf/1812.01187.pdf
    return lambda x: ((1 - math.cos(x * math.pi / steps)) / 2) * (y2 - y1) + y1


linear_lr = False
epochs = 10
if linear_lr:
    lf = lambda x: (1 - x / (epochs - 1)) * (1.0 - hyp['lrf']) + hyp['lrf']
else:
    lf = one_cycle(1, hyp['lrf'], epochs)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)

In [39]:
if ckpt['optimizer'] is not None:
    optimizer.load_state_dict(ckpt['optimizer'])
    best_fitness = ckpt['best_fitness']

start_epoch = ckpt['epoch'] + 1

del ckpt

In [41]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset, 
    batch_size=batch_size,
    num_workers=4,
    sampler=None,
    pin_memory=True,
    collate_fn=LoadImagesAndLabels.collate_fn
)

## Hyperparameters

In [46]:
nl = model.model[-1].nl
hyp['box'] *= 3 / nl  # scale to layers
hyp['cls'] *= nc / 80 * 3 / nl  # scale to classes and layers
hyp['obj'] *= (IMG_SIZE / 640) ** 2 * 3 / nl  # scale to image size and layers
hyp['label_smoothing'] = False
model.nc = nc
model.hyp = hyp
model.names = data['names']

## Loss

In [47]:
from utils.loss import ComputeLoss

compute_loss = ComputeLoss(model)

## Train

In [51]:
nw = 1
nb = len(train_dataloader)

In [55]:
from torch.cuda import amp

cuda = not (device == 'cpu')
last_opt_step = -1

for epoch in range(start_epoch, epochs):
    print(epoch)
    model.train()
    
    mloss = torch.zeros(3, device=device)
    optimizer.zero_grad()
    
    for i, (imgs, targets, paths, boxes) in enumerate(train_dataloader):
        ni = i + nb * epoch
        imgs = imgs.to(device, non_blocking=True).float() / 255.0
        
        # Warmup
        if ni <= nw:
            xi = [0, nw]  # x interp
            # compute_loss.gr = np.interp(ni, xi, [0.0, 1.0])  # iou loss ratio (obj_loss = 1.0 or iou)
            accumulate = max(1, np.interp(ni, xi, [1, nbs / batch_size]).round())
            for j, x in enumerate(optimizer.param_groups):
                # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                x['lr'] = np.interp(ni, xi, [hyp['warmup_bias_lr'] if j == 2 else 0.0, x['initial_lr'] * lf(epoch)])
                if 'momentum' in x:
                    x['momentum'] = np.interp(ni, xi, [hyp['warmup_momentum'], hyp['momentum']])
        
        # Forward
        with amp.autocast(enabled=cuda):
            pred = model(imgs)  # forward
            loss, loss_items = compute_loss(pred, targets.to(device))  # loss scaled by batch_size
            print(loss)
        
        loss.backward()
        
        # optimize
        if ni - last_opt_step >= accumulate:
            print("Optimizing")
            optimizer.step()
            optimizer.zero_grad()
            last_opt_step = ni
    
    lr = [x['lr'] for x in optimizer.param_groups]  # for loggers
    scheduler.step()

0


/home/isabella/code/competitions/great_barrier_reef/env/lib/python3.7/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


tensor([1.09261], grad_fn=<MulBackward0>)
Optimizing


/home/isabella/code/competitions/great_barrier_reef/env/lib/python3.7/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


tensor([1.00333], grad_fn=<MulBackward0>)


...

/home/isabella/code/competitions/great_barrier_reef/env/lib/python3.7/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


tensor([0.65789], grad_fn=<MulBackward0>)
Optimizing


/home/isabella/code/competitions/great_barrier_reef/env/lib/python3.7/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


tensor([0.93230], grad_fn=<MulBackward0>)


KeyboardInterrupt: 

In [58]:
accumulate

8.0

# Pytorch lightning

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer

In [ ]:
from pytorch_l

In [74]:
from reef.yolo.lightning import YOLOModel

module = (
    YOLOModel
    .load_from_pretrained(
        "../../weights/yolov5s.pt",
        hyp=hyp,
        data=data,
        random_state=42
    )
    .load_train_params(batch_size=4)
)
module

12/16/2021 14:15:03 - INFO - utils.general -   Overriding model.yaml nc=80 with nc=1
12/16/2021 14:15:03 - INFO - utils.general -   Overriding model.yaml anchors with anchors=3
12/16/2021 14:15:03 - INFO - utils.general -   
                 from  n    params  module                                  arguments                     
12/16/2021 14:15:03 - INFO - utils.general -     0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
12/16/2021 14:15:04 - INFO - utils.general -     1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
12/16/2021 14:15:04 - INFO - utils.general -     2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
12/16/2021 14:15:04 - INFO - utils.general -     3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
12/16/2021 14:15:04 - INFO - utils.general -     4  

YOLOModel(
  (model): Model(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=